In [1]:
import json
from IPython.display import display, Image
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import Image as WidgetImage

In [2]:
def load_data(filename):
    with open(filename, 'r') as file:
        return json.load(file)

In [3]:
def save_updated_data(updated_data, start, end):
    with open(f'MB_{start}_{end}.json', 'w') as file:
        json.dump(updated_data, file)

In [28]:
class label_tool():
    def __init__(self, json_name, start, end):
        self.json_name = json_name
        self.current_index = start
        self.data = load_data(json_name) 
        self.end = end
        self.start = start
        self.reset_button = widgets.Button(description="Reset")
        self.save_next_button = widgets.Button(description="Save and Next")
        self.question_text = widgets.Text(description="Question:",layout={'width': '80%'})
        self.type_buttons = widgets.RadioButtons(options=['Size', 'Color', 'Texture', 'Shape', 'Pattern', 'State', 'None'], value='None', description='Type:')
        self.answer_buttons = widgets.RadioButtons(options=['Left', 'Right', 'None'], value='None', description='Answer:')
        self.delete_buttons = widgets.Button(description="Delete")
        
    def display_content(self):
        clear_output()
        if self.current_index < self.end:
            entry = self.data[self.current_index]
            if not entry:
                print('empty')
                self.current_index += 1
                self.display_content()
            # Load images from files and display them side by side
            with open(entry['image_1'], "rb") as file:
                image_input = file.read()
            with open(entry['image_2'], "rb") as file:
                image_output = file.read()
            

            print(f' Image ID {self.current_index}')
            
            box = widgets.HBox([
                WidgetImage(value=image_input, format='png', width=250, height=250),
                WidgetImage(value=image_output, format='png', width=250, height=250)
            ])

    
            self.question_text.value = self.data[self.current_index]['question']
            self.answer_buttons.value = self.data[self.current_index]['answer']
            self.type_buttons.value = self.data[self.current_index]['type']
            display(box, self.question_text, self.type_buttons, self.answer_buttons, self.save_next_button, self.delete_buttons)


        else:
            print("No more entries to display. Please call save_updated_data() to save results.")

    def save_and_next(self, button):
        """Save paths, type, answer, and update the question."""

        
        data = {'image_1': self.data[self.current_index]['image_1'], 'image_2': self.data[self.current_index]['image_2'], 
                             'type': self.type_buttons.value, 
                             'answer': self.answer_buttons.value, 
                             'question': self.question_text.value}
        print(data)
        self.data[self.current_index] = data
        self.current_index += 1
        self.display_content()
        
    
    def begin(self):
        if not self.data[self.current_index]:
            print('empty')
            self.current_index += 1
            self.display_content()
        self.question_text.value = self.data[self.current_index]['question']
        # Link buttons to functions
        self.save_next_button.on_click(self.save_and_next)
        self.delete_buttons.on_click(self.delete_function)
        self.display_content()
        
    def delete_function(self, button):
        self.data[self.current_index] = {}
        print(f'remove {self.current_index} pair')
        self.current_index += 1
        self.display_content()
        
    def save_updated_data(self):
        with open(self.json_name, 'w') as file:
            json.dump(self.data, file)
            


In [29]:
labeler = label_tool('State_Transform_0_200.json', 0,120)

In [30]:
labeler.begin()

In [18]:
labeler.data[0]

In [10]:
labeler.save_updated_data()